# Coursera Capstone Project
## This notebook will be used primarily for the capstone project. ..




In [52]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests 

In [4]:
print("Hello Capstone Project Course")

Hello Capstone Project Course


## Week 3: Segmenting and Clustering Neighborhoods in Toronto

In [102]:
#grab URL and store it in a URL variable
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#obtain raw html
r = requests.get(URL)
#parse into beautifulsoup 
soup = BeautifulSoup(r.content) 
#find required content
table = soup.findAll('table',attrs ={'rules':'all','cellspacing':0,}) 
#convert to text
a = table[0].text
b = a.split("\n")
for word in b:
    word.replace("\n", "")
    word.replace("\n", "")
    word.replace("\n", "")
c = []
for word in b:
    if "Not assigned" not in word and word != "":
        c.append(word)
table_contents=[]
for word in c:
    cell = {}
    cell['PostalCode'] = word[:3]
    temp = word.replace(word[:3], '')
    cell['Borough'] = temp.split('(')[0]
    cell['Neighborhood'] = (((((temp).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
    table_contents.append(cell)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

    
    
    

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [103]:
df.shape

(103, 3)